<a href="https://colab.research.google.com/github/Chalita20/Age-classification-by-using-the-AFAD-dataset-Internship/blob/main/Age_classification_by_using_images_of_the_AFAD_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference1 : https://medium.com/analytics-vidhya/implementing-cnn-in-pytorch-with-custom-dataset-and-transfer-learning-1864daac14cc

reference2 : https://nextjournal.com/gkoehler/pytorch-mnist

# Age classification by using images of the AFAD dataset 

# ขั้นตอนการทำงาน

* การเชื่อมกับ Drive เพื่อใช้ในการดึงข้อมูล
  * การ unzip และการเข้าถึงโฟลเดอร์
* การเตรียม Dataset 
  * การเตรียมไฟล์ csv
  * การ import library ที่ต้องใช้ในการtrain โมเดล
  * การแบ่ง dataset
* การเตรียม model 
  * การสร้าง network สำหรับtrain model
  * การtrain model
  * การ test model
* การทำนายผลของmodel ที่ได้
  * การแสดงค่าAccuracy และค่าLoss
  * การแสดงผลแบบรูปภาพ


## Mount Drive

เป็นการเชื่อมต่อกับไดร์ฟเพื่อดึงข้อมูลจากไดร์ฟมาใช้งานโดยต้องเชื่อมไดร์ฟก่อนทุกครั้ง
* เชื่อมกับไดร์ฟที่ต้องการ
* เข้าถึงโฟลเดอร์ที่ต้องการ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

In [ ]:
%cd drive/My Drive/Colab Notebooks/Dataset/

ขั้นตอนต่อไปคือจะต้องโคลนข้อมูลหรือโปรเจคที่เราต้องการใช้จากgithub มาไว้ในไดร์ฟของเรา เราจะทำการโคลนแค่ครั้งแรกตอนที่เราจะดึงข้อมูลเข้าไดร์ฟ

In [ ]:
# !git clone https://github.com/afad-dataset/tarball-lite.git # รันแค่ครั้งแรกอย่างเดียว

### unzip folder images 
1. จะต้องแปลงไฟล์zip ให้เป็นไฟล์ .tar.xz 
2. จะต้องแตกไฟล์zip .tar.xz หลังจากนั้นก็สามารถดึงข้อมูลdataset จากgithubได้

** โดยการแตกไฟล์จะรันแค่ครั้งแรกครั้งเดียว เพราะจะทำให้ข้อมูลซ้ำกัน **

In [ ]:
# !cat AFAD-Lite.tar.xz* > AFAD-Lite.tar.xz # รันแค่ครั้งแรกอย่างเดียว

In [ ]:
# !tar -xf AFAD-Lite.tar.xz # รันแค่ครั้งแรกอย่างเดียว

เข้าโฟลเดอร์รูปภาพที่แตกไฟล์ เพื่อจะดึงรูปมาใช้

In [ ]:
%cd ./tarball-lite/AFAD-Lite/

## การเตรียม Dataset 

### การเตรียมไฟล์ csv
* แบบที่ 1 แบบ labels ธรรมดา
* แบบที่ 2 แบบOne-hot encoding


**สามารถเลือกแบบใดแบบหนึ่งได้**

ก่อนที่เราจะทำไฟล์เราจะต้องดูว่าเราเข้าโฟลเดอร์รูปภาพหรือโฟลเดอร์ที่ไฟล์เราอยู่ยังเพื่อที่จะได้ดึงข้อมูลเพราะถ้ายังไม่ได้อยู่ในโฟลเดอร์จะทำให้อาจจะหาไฟล์ไม่เจอ

การดึงข้อมูลรูปภาพที่อยู่แบบsub-folder แล้วทำเป็นไฟล์csv 
สามารถนำไฟล์csvที่ได้ไปใช้ได้เลยแต่จะมีจำนวนclassตามโฟลเดอร์ที่มีอยู่

In [ ]:
import numpy as np
import pandas as pd
import os,sys
import csv
import glob
import os.path

data_path="/content/drive/My Drive/Colab Notebooks/Dataset/tarball-lite/AFAD-Lite" #ให้ใส่pathของรูปภาพที่ต้องการ
with open('dataset_New_1.csv', 'w', newline='') as f:
        fieldnames = ['file_name', 'age']
        thewriter = csv.DictWriter(f, fieldnames=fieldnames)
        thewriter.writeheader()
        for path, dirs, files in os.walk(data_path):
            for i in files:
                n = os.path.join(path,i)
                x = n.split(os.sep)
                thewriter.writerow({'file_name': n, 'age':x[8]})

#### แบบ labels ที่ทำเป็นแบบ 5 class

In [ ]:
import pandas as pd

df1 = pd.read_csv('dataset_New_1.csv')
df1

In [ ]:
df2 = df1.drop(df1.index[0])
df3 = df2.sort_values(by='age')  # การเรียงข้อมูลตามคอลัมน์ของage

In [ ]:
z = []
for x in df3["age"]:
  if int(x) >=18 and int(x) <=21:
    z.append(1)
  elif int(x) >=22 and int(x) <=25:
    z.append(2)
  elif int(x) >=26 and int(x) <=29:
    z.append(3)
  elif int(x) >=30 and int(x) <=34:
    z.append(4)
  elif int(x) >=35 and int(x) <=39:
    z.append(5)

df3["labels"] = z
df3

In [ ]:
df4 = df3.drop(columns='age') #ลบcolumnsที่เกินออก

In [ ]:
df4.to_csv('New_Dataset_1.csv') #saveไฟล์csvอันใหม่

#### แบบ One-Hot encoding ที่ทำเป็นแบบ 5 class

In [ ]:
import pandas as pd

df1 = pd.read_csv('dataset_New_1.csv') #เรียกไฟล์csvจากการใส่pathและlabels ของ age
df2 = df1.drop(df1.index[0])
df3 = df2.sort_values(by='age')

In [ ]:
z = []
for x in df3["age"]:
  if int(x) >=18 and int(x) <=21:
    z.append(1)
  elif int(x) >=22 and int(x) <=25:
    z.append(2)
  elif int(x) >=26 and int(x) <=29:
    z.append(3)
  elif int(x) >=30 and int(x) <=34:
    z.append(4)
  elif int(x) >=35 and int(x) <=39:
    z.append(5)

df3["labels"] = z
df3

In [ ]:
df4 = df3.drop(columns='age')
cat_data = df4[['file_name']]
num_data = df4[['labels']]

In [ ]:
top = [x for x in df4.labels.value_counts().sort_index(ascending=True).index]
top

In [ ]:
import numpy as np

for label in top:
  df4[label] = np.where(df4['labels']==label, 1, 0)

df5 = df4[['labels']+top]

In [ ]:
df = df5.drop(columns=['labels'])
dataset = pd.concat([cat_data, df],axis=1)

In [ ]:
dataset.to_csv('New_Dataset_1.csv')

### Importing Libraries 

ก่อนที่เราจะทำdataset เราต้องทำการ import library ที่เราจะต้องใช้งานมาด้วย

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
import torch.nn.functional as F
import pandas as pd
import os
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.autograd import Variable
from PIL import Image
import numpy as np 

device = ("cuda" if torch.cuda.is_available() else "cpu")

### Preparing the Dataset

In [ ]:
n_epochs = 30
batch_size = 32
learning_rate = 0.001
shuffle = True
pin_memory = True
num_workers = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

#### Creating custom Dataset

In [ ]:
class FaceDataset(Dataset):
  def __init__(self, root_dir, annotation_file, transform=None):
    self.root_dir = root_dir
    self.annotation = pd.read_csv(annotation_file)
    self.transform = transform

  def __len__(self):
    return len(self.annotation)

  def __getitem__(self, index):
    img_id = self.annotation.iloc[index, 1]
    img = Image.open(os.path.join(self.root_dir, str(img_id))).convert("RGB")
    y_label = self.annotation.iloc[index, 2:]
    y_label = y_label.astype("float", copy=False)
    y_label = torch.tensor(y_label)
    if self.transform:
      img = self.transform(img)
    
    return (img, y_label)

#### Transformations

การtransform เพื่อทำให้inputของเรามีขนาดที่ตรงตามโมเดลกำหนดได้

In [ ]:
transform = transforms.Compose([
        transforms.Resize((120, 120)),
        # transforms.RandomCrop((299,299)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

### Setting the dataset and dataloader

เป็นการแบ่งdatasetของเราเป็น 3ส่วน
* train ใช้สำหรับการป้อนให้กับNetwork เพื่อใช้ trainโมเดล
* validation ใช้สำหรับทดสอบหา Metries หลังจากtrainเสร็จว่าโมเดล ทำงานได้ดีแค่ไหน และเช็คแต่ละครั้งของโมเดลไหนทำงานได้ดีกว่ากัน
* test ใช้สำหรับทดสอบหลังจากได้โมเดลที่ดีที่สุด ว่าโมเดลจะทำงานได้ดีแค่ไหนกับข้อมูลที่ไม่เคยเห็น

** โดยการแบ่งจะทำการสุ่มข้อมูลออกเป็น3 ส่วนตามอัตราส่วนของข้อมูลที่เรากำหนด

In [ ]:
dataset = FaceData("train", "Dataset_2.csv", transform=transform)
lenghts = [int(round(len(dataset)*0.7)),int(round(len(dataset)*0.2)),
           int(round(len(dataset)*0.1))]
train_set, validation_set, test_set = torch.utils.data.random_split(dataset, lenghts)
train_loader = DataLoader(dataset=train_set, shuffle=shuffle,
                          batch_size=batch_size, num_workers=num_workers,
                          pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle,
                               batch_size=batch_size,num_workers=num_workers,
                               pin_memory=pin_memory)
test_loader = DataLoader(dataset=test_set, shuffle=shuffle,
                          batch_size=batch_size, num_workers=num_workers,
                          pin_memory=pin_memory)

#### Examples from folder

เอาไว้ลองเช็ครูปภาพในโฟลเดอร์ที่เราทำการแบ่งไว้

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
  plt.show()
fig

## Model

การเตรียมNetworkเพื่อจะเริ่ม train model  เป็นการประกาศ class ของ network เพื่อที่จะเรียกไปใช้งานตอนtrain

### Building the network

In [ ]:
seed=0
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

สร้างคลาสของ network LeNet โดยมีโครงสร้างดังนี้

* First Layer
	> input: (120, 120, 3) \
  > after padding: (124, 124, 3) \
  >  after convolution(kernel = 5*5): (120, 120, 16) where 124-5+1 = 120\
  >  after ReLU: (120, 120, 16)\
  >  after maxpooling(stride=2*2): (60, 60, 16) >> output 
* Second Layer
	> input: (60, 60, 16) \
	> after convolution(kernel = 5*5): (56, 56, 32)\
	> after ReLU: (56, 56, 32)\
	> after maxpooling(stride=2*2): (28, 28, 32) >> output
* Third Layer
	> input: (32, 28, 28)\
	> after fully-connected: (120, 3)\
	> after ReLU: (120, 3) > output 
* Fourth Layer
	> input: (120, 3)\
	> after fully-connected: (84, 3)\
	> after ReLU: (84, 3) > output
* Fifth Layer
	  input: (84, 3)

	  after fully-connected: (5, 3)
	  after ReLU: (5, 3) > output

> self.conv1 = nn.Conv2d(3, 16, 5, padding=2)\
> self.conv2 = nn.Conv2d(16, 32, 5)\
> self.conv3 = nn.Conv2d(32, 64, 5)\
> self.fc1 = nn.Linear(64*28*28, 120)\
> self.fc2   = nn.Linear(120, 84)\
> self.fc3   = nn.Linear(84, 5) 


In [ ]:
class LeNet(nn.Module):

    # network structure
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1   = nn.Linear(32*28*28, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 5)

    def forward(self, x):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, 32*28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

ส่วนนี้คือการดาวน์โหลดnetworkที่ประกาศไว้มาใช้งาน 
โดย networkที่ใช้คือ LeNet (เราสามารถเรียกNetworkอื่นที่เหมาะสมกว่าได้)

In [ ]:
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training the model

การtrainโมเดลจะทำการทดสอบด้วยvalidation ทุกรอบของการtrainและจะsave model ทุกรอบของการtrain

In [ ]:
def train():
    model.train()
    for epoch in range(n_epochs):
      training_loss = 0.0
      val_loss = 0.0
      val_acc = 0
      correct_preds = 0
      correct_train = 0
      best_acc = 0
      validation = 0.0
      total = 0
      total_train_acc = 0

      loop = tqdm(enumerate(train_loader), total = len(train_loader), leave=True)
      
      for batch_idx, (x_imgs, y_labels) in loop:
        x_imgs = x_imgs.to(device)
        y_labels = y_labels.to(device)
        outputs = model(x_imgs)

        _, predicted = torch.max(outputs, 1)
        _, lalabel = torch.max(y_labels.data, 1)
        correct_train += (predicted == lalabel).sum().item()
        total_train_acc += y_labels.size(0)
        
        loss = criterion(outputs.float(), torch.max(y_labels, 1)[1])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()

      valid_data_loader = tqdm(validation_loader, total=len(validation_loader), leave=True)

      with torch.no_grad():
        model.eval()
        for x_imgs, y_labels in valid_data_loader:
            x_imgs = x_imgs.to(device)
            y_labels = y_labels.to(device)
            outputs = model(x_imgs)

            val_loss = criterion(outputs, torch.max(y_labels, 1)[1])
            validation += val_loss.item()

            total += y_labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            _, lalabel = torch.max(y_labels.data, 1)
            correct_preds += (predicted == lalabel).sum().item()

            torch.save(model.state_dict(), 'model_27_4_64.pth')
            torch.save(optimizer.state_dict(), 'optimizer_27_4_64.pth')

        train_acc = 100 * (correct_train / total_train_acc)
        training_loss /= len(train_loader.dataset)
        validation /= len(validation_loader.dataset)
        val_acc = 100* (correct_preds / total)

        print(f'Epoch: {epoch+1}/{n_epochs}.. Training loss is: {training_loss}.. Validation Loss: {validation}')
        print('Training Accuracy is: {:.2f}%'.format(train_acc))
        print('Validation Accuracy is: {:.2f}%'.format(val_acc))

if __name__ == "__main__":
    train()

### continued model

เมื่อเราต้องการที่จะเทรนโมเดลต่อเราสามารถทำขั้นตอนข้างล่างต่อได้เลย โดยเป็นการเรียนweightของโมเดลที่เราเทรนไปบางส่วนแล้วมาทำการเทรนและจะมีการsave weight ใหม่ทุกๆการรันในแต่ละรอบ

**โดยขั้นตอนนี้สามารถทำหรือไม่ทำก็ได้แล้วแต่ความพึงพอใจของโมเดลที่เราได้**

In [ ]:
continued_model = LeNet().to(device)
continued_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model_state_dict = torch.load('model_24_4_21.pth')
continued_model.load_state_dict(model_state_dict)

optimizer_state_dict = torch.load('optimizer_24_4_21.pth')
continued_optimizer.load_state_dict(optimizer_state_dict)

In [ ]:
def train():
    model.eval()
    for epoch in range(11, 20):
      training_loss = 0.0
      val_loss = 0.0
      val_acc = 0
      correct_preds = 0
      correct_train = 0
      best_acc = 0
      validation = 0.0
      total = 0
      total_train_acc = 0

      loop = tqdm(enumerate(train_loader), total = len(train_loader), leave=True)
      
      for batch_idx, (x_imgs, y_labels) in loop:
        x_imgs = x_imgs.to(device)
        y_labels = y_labels.to(device)
        outputs = model(x_imgs)

        _, predicted = torch.max(outputs, 1)
        _, lalabel = torch.max(y_labels.data, 1)
        correct_train += (predicted == lalabel).sum().item()
        total_train_acc += y_labels.size(0)
        
        loss = criterion(outputs.float(), torch.max(y_labels, 1)[1])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()

      valid_data_loader = tqdm(validation_loader, total=len(validation_loader), leave=True)

      with torch.no_grad():
        model.eval()
        for x_imgs, y_labels in valid_data_loader:
            x_imgs = x_imgs.to(device)
            y_labels = y_labels.to(device)
            outputs = model(x_imgs)

            val_loss = criterion(outputs, torch.max(y_labels, 1)[1])
            validation += val_loss.item()

            total += y_labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            _, lalabel = torch.max(y_labels.data, 1)
            correct_preds += (predicted == lalabel).sum().item()

            torch.save(model.state_dict(), 'model_24_4_21.pth')
            torch.save(optimizer.state_dict(), 'optimizer_24_4_21.pth')

        train_acc = 100 * (correct_train / total_train_acc)
        training_loss /= len(train_loader.dataset)
        validation /= len(validation_loader.dataset)
        val_acc = 100* (correct_preds / total)

        print(f'Epoch: {epoch+1}/{n_epochs}.. Training loss is: {training_loss}.. Validation Loss: {validation}')
        print('Training Accuracy is: {:.2f}%'.format(train_acc))
        print('Validation Accuracy is: {:.2f}%'.format(val_acc))

if __name__ == "__main__":
    train()

#### test model

การนำเอาโมเดลที่save แล้วมาทดสอบต่อด้วยโฟลเดอร์ของtest จึงดึงโมเดลอีกรอบ 

In [ ]:
import numpy as np
from misc import progress_bar 

test_loss = 0.0
total     = 0.0
test_correct =0.0

for i, (x_imgs, y_labels) in enumerate(test_loader, 0):
  ### Extract image and labels from trainloader
  x_imgs  = x_imgs.to(device)
  y_labels  = y_labels.to(device)

  outputs = model(x_imgs)

  loss    = criterion(outputs, torch.max(y_labels, 1)[1])
  test_loss += loss.item()

  ### select class index with the maximum value 
  total     += y_labels.size(0)
  _, prediction = torch.max(outputs, 1)
  _, lalabel = torch.max(y_labels, 1)
  
  # correct_preds += (predicted == lalabel).sum().
  test_correct += (prediction==lalabel).sum().item()
  progress_bar(i, len(test_loader), 'Loss: %.4f | Acc: %.3f%% (%d/%d)'
                            % (test_loss / (i + 1), 100. * test_correct / total, test_correct, total))
  

print('test_loss = '+str(test_loss/len(test_loader.dataset)))
print('test_accuracy = '+ str(test_correct/total))


## Prediction

เป็นการเรียกแสดงผลรูปจากโฟลเดอร์ test set ว่ามีการทำนายรูปได้ถูกต้องหรือไม่โดยการแสดงผลเป็นทั้งรูปภาพและผลการทำนาย

In [ ]:
with torch.no_grad():
  # model.eval()
  # exampl_data = example_data.to(device)
  output = model(example_data.to(device))

In [ ]:
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title(f'Prediction: {output.data.max(1, keepdim=True)[1][i].item()}')
  plt.xticks([])
  plt.yticks([])
  plt.show()
fig

เป็นการtest รูปจากข้างนอกเพื่อดูผลความแม่นยำของModelของเราว่ามีผลที่ถูกต้องหรือไม่

In [ ]:
model.train()
transform = transforms.Compose([
        transforms.Resize((120, 120)),
        # transforms.RandomCrop((299,299)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

img = Image.open('check_image.jpg')  # Load image as PIL.Image
x = transform(img)  # Preprocess image
x = x.unsqueeze(0)  # Add batch dimension


output = model(x.to(device=device))  # Forward pass'
pred = torch.argmax(output, 1)  # Get predicted class if multi-class classification
print('Image predicted as ', pred)